In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype
import warnings
import pickle
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import BernoulliNB
from lightgbm import LGBMClassifier
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
import itertools
from tabulate import tabulate
import joblib
from joblib import dump


In [3]:
train=pd.read_csv('./dataset/trainData.txt', sep=',')
test=pd.read_csv('./dataset/testData.txt',sep=',')

In [4]:
train.head(100)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2,tcp,smtp,SF,3065,331,0,0,0,0,...,119,0.48,0.02,0.00,0.02,0.01,0.02,0.00,0.00,normal
96,0,udp,other,SF,102,102,0,0,0,0,...,1,0.14,0.29,0.14,0.00,0.00,0.00,0.00,0.00,normal
97,0,tcp,http,SF,259,750,0,0,0,0,...,255,1.00,0.00,0.25,0.03,0.00,0.00,0.00,0.00,normal
98,1082,udp,other,SF,147,105,0,0,0,0,...,2,0.01,0.42,0.86,0.00,0.00,0.00,0.00,0.00,normal


In [5]:
train.describe(include='object')

,protocol_type,service,flag,class
count,125973,125973,125973,125973
unique,3,70,11,2
top,tcp,http,SF,normal
freq,102689,40338,74945,67343


In [6]:
# train.isnull().sum()

In [7]:
total=train.shape[0]
missing_columns= [col for col in train.columns if train[col].isnull().sum()>0]
for col in missing_columns:
    null_count = train[col].isnull().sum()
    per= (null_count/total)*100
    print(f"{col}:{null_count} ({roun(per,3)}%)")

In [8]:
print(f"Number of duplicate rows: {train.duplicated().sum()}")

Number of duplicate rows: 9


In [9]:
# sns.countplot(x=train['protocol_type'])

In [10]:
# sns.countplot(x=train['class'])

In [11]:
# sns.countplot(x=test['class'])

In [12]:
# sns.countplot(x=train['service'])

In [13]:
print("Class distribution training set :")
print(train['class'].value_counts())

Class distribution training set :
normal     67343
anomaly    58630
Name: class, dtype: int64


In [14]:
train

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,tcp,private,S0,0,0,0,0,0,0,...,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
125969,8,udp,private,SF,105,145,0,0,0,0,...,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal
125970,0,tcp,smtp,SF,2231,384,0,0,0,0,...,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal
125971,0,tcp,klogin,S0,0,0,0,0,0,0,...,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly


In [15]:
def le(df):
    for col in df.columns:
        if df[col].dtype=='object':
            label_encoder = LabelEncoder()
            df[col]= label_encoder.fit_transform(df[col])
le(train)
le(test)

In [16]:
train

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,1,20,9,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,1
1,0,2,44,9,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,1
2,0,1,49,5,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,0
3,0,1,24,9,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,1
4,0,1,24,9,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,1,49,5,0,0,0,0,0,0,...,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,0
125969,8,2,49,9,105,145,0,0,0,0,...,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,1
125970,0,1,54,9,2231,384,0,0,0,0,...,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,1
125971,0,1,30,5,0,0,0,0,0,0,...,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,0


In [17]:
train.drop(['num_outbound_cmds'],axis=1,inplace=True)
test.drop(['num_outbound_cmds'],axis=1,inplace=True)

In [18]:
train.head(89)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,1,20,9,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,1
1,0,2,44,9,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,1
2,0,1,49,5,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,0
3,0,1,24,9,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,1
4,0,1,24,9,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,0,1,24,9,320,379,0,0,0,0,...,255,1.00,0.00,0.01,0.02,0.00,0.00,0.00,0.00,1
85,0,1,49,5,0,0,0,0,0,0,...,7,0.10,0.11,0.01,0.00,0.97,1.00,0.00,0.00,0
86,0,1,24,9,225,3762,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1
87,0,1,19,5,0,0,0,0,0,0,...,1,0.00,0.07,0.00,0.00,1.00,1.00,0.00,0.00,0


In [19]:
x_train = train.drop(['class','duration','land' ,'wrong_fragment' ,'urgent','hot','num_failed_logins' ,'logged_in','num_compromised','root_shell','su_attempted' ,'num_root' ,'num_file_creations' ,'num_shells','num_access_files','is_host_login','is_guest_login' ,'srv_count','serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate','srv_diff_host_rate','dst_host_count','dst_host_diff_srv_rate' ,'dst_host_same_src_port_rate','dst_host_srv_diff_host_rate','dst_host_serror_rate' ,'dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate'],axis=1)
y_train =train['class']

In [20]:
service =train['service']
service

0         20
1         44
2         49
3         24
4         24
          ..
125968    49
125969    49
125970    54
125971    30
125972    20
Name: service, Length: 125973, dtype: int32

In [21]:
rfc = RandomForestClassifier()

rfe = RFE(rfc,n_features_to_select=10)
rfe= rfe.fit(x_train,y_train)

feature_map=[(i,v) for i,v in itertools.zip_longest(rfe.get_support(),x_train.columns)]
selected_features = [v for i, v in feature_map if i==True]

selected_features

['protocol_type',
 'service',
 'flag',
 'src_bytes',
 'dst_bytes',
 'count',
 'same_srv_rate',
 'diff_srv_rate',
 'dst_host_srv_count',
 'dst_host_same_srv_rate']

In [22]:
x_train=x_train[selected_features]
print(x_train)

        protocol_type  service  flag  src_bytes  dst_bytes  count  \
0                   1       20     9        491          0      2   
1                   2       44     9        146          0     13   
2                   1       49     5          0          0    123   
3                   1       24     9        232       8153      5   
4                   1       24     9        199        420     30   
...               ...      ...   ...        ...        ...    ...   
125968              1       49     5          0          0    184   
125969              2       49     9        105        145      2   
125970              1       54     9       2231        384      1   
125971              1       30     5          0          0    144   
125972              1       20     9        151          0      1   

        same_srv_rate  diff_srv_rate  dst_host_srv_count  \
0                1.00           0.00                  25   
1                0.08           0.15               

In [23]:
scale= StandardScaler()
x_train=scale.fit_transform(x_train)
test= scale.fit_transform(test)

In [24]:
X_train,X_test,Y_train,Y_test =train_test_split(x_train,y_train,train_size=0.70,random_state=2)

In [25]:
import time
from sklearn.linear_model import LogisticRegression

clfl=LogisticRegression(max_iter=1200000)
start_time=time.time()
clfl.fit(X_train,Y_train.values.ravel())
end_time=time.time()
print("Training time: ",end_time-start_time)

Training time:  0.7544772624969482


In [26]:
start_time=time.time()
Y_test_pred = clfl.predict(X_train)
end_time=time.time()
print("Testing time: ",end_time-start_time)

Testing time:  0.028745174407958984


In [27]:
lg_model= LogisticRegression(random_state=42)
lg_model.fit(X_train,Y_train)

LogisticRegression(random_state=42)

In [28]:
lg_train,lg_test=lg_model.score(X_train,Y_train) , lg_model.score(X_test,Y_test)

print(f"Training score: {lg_train}")
print(f"Test score: {lg_test}")

Training score: 0.9289983102936007
Test score: 0.9273126587637596


In [29]:
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [30]:
def objective(trial):
    n_neighbors=trial.suggest_int('KNN_n_neighbors',2,16,log=False)
    classifier_obj=KNeighborsClassifier(n_neighbors=n_neighbors)
    classifier_obj.fit(X_train,Y_train)
    accuracy=classifier_obj.score(X_test,Y_test)
    return accuracy

In [31]:
study_KNN = optuna.create_study(direction='maximize')
study_KNN.optimize(objective,n_trials=1)
print(study_KNN.best_trial)

FrozenTrial(number=0, state=TrialState.COMPLETE, values=[0.9865315410668924], datetime_start=datetime.datetime(2024, 3, 24, 17, 43, 15, 222300), datetime_complete=datetime.datetime(2024, 3, 24, 17, 43, 48, 834130), params={'KNN_n_neighbors': 12}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'KNN_n_neighbors': IntDistribution(high=16, log=False, low=2, step=1)}, trial_id=0, value=None)


In [32]:
KNN_model = KNeighborsClassifier(n_neighbors=study_KNN.best_trial.params['KNN_n_neighbors'])
KNN_model.fit(X_train,Y_train)

KNN_train,KNN_test= KNN_model.score(X_train,Y_train),KNN_model.score(X_test,Y_test)

print(f"Training score: {KNN_train}")
print(f"Test score: {KNN_test}")

Training score: 0.9878205055510825
Test score: 0.9865315410668924


In [33]:
from sklearn.tree import DecisionTreeClassifier

clfd =DecisionTreeClassifier(criterion='entropy',max_depth=4)
start_time= time.time()
clfd.fit(X_train,Y_train.values.ravel())
end_time=time.time()
print("Training time: ",end_time-start_time)

Training time:  0.3980519771575928


In [34]:
start_time= time.time()
Y_test_pred=clfd.predict(x_train)
end_time=time.time()
print("Testing time: ",end_time-start_time)

Testing time:  0.03240847587585449


In [35]:
def objective(trial):
    dt_max_depth=trial.suggest_int('dt_max_depth',2,32,log=False)
    dt_max_features=trial.suggest_int('dt_max_features',2,10,log=False)
    classifier_obj=DecisionTreeClassifier(max_features=dt_max_features,max_depth=dt_max_depth)
    classifier_obj.fit(X_train,Y_train)
    accuracy=classifier_obj.score(X_test,Y_test)
    return accuracy

In [36]:
study_dt = optuna.create_study(direction='maximize')
study_dt.optimize(objective,n_trials=30)
print(study_dt.best_trial)

FrozenTrial(number=13, state=TrialState.COMPLETE, values=[0.9980948348856901], datetime_start=datetime.datetime(2024, 3, 24, 17, 45, 3, 363038), datetime_complete=datetime.datetime(2024, 3, 24, 17, 45, 3, 979274), params={'dt_max_depth': 18, 'dt_max_features': 10}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'dt_max_depth': IntDistribution(high=32, log=False, low=2, step=1), 'dt_max_features': IntDistribution(high=10, log=False, low=2, step=1)}, trial_id=13, value=None)


In [37]:
dt =DecisionTreeClassifier(max_features=study_dt.best_trial.params['dt_max_features'],max_depth=study_dt.best_trial.params['dt_max_depth'])
dt.fit(X_train,Y_train)
dump(dt,'crssri.joblib')
dt_train,dt_test = dt.score(X_train,Y_train), dt.score(X_test,Y_test)

print(f"Training score: {dt_train}")
print(f"Test score: {dt_test}")

Training score: 0.9999092775087604
Test score: 0.997962531752752


In [38]:
X_train

array([[ 2.21931185, -1.17618945,  0.75111129, ..., -0.29422413,
         1.22262133,  1.02185272],
       [-0.12470616,  1.08730518, -0.73623464, ..., -0.0169296 ,
        -0.89115586, -1.00510998],
       [-0.12470616,  1.63788495, -0.73623464, ..., -0.0723885 ,
        -1.03568764, -1.16103019],
       ...,
       [-0.12470616,  0.29202328, -2.22358057, ...,  0.03852931,
        -0.82792321, -0.96056135],
       [ 2.21931185,  1.08730518,  0.75111129, ...,  0.20490602,
        -0.85502292, -0.98283566],
       [ 2.21931185,  0.78142753,  0.75111129, ..., -0.34968303,
        -1.01762117, -1.13875587]])

In [39]:
data =[
    ["KNN",KNN_train,KNN_test],
    ["Logistic Regression", lg_train,lg_test],
    ["Decision Tree",dt_train,dt_test]
]

col_names= ["Model","Train score","Test score"]
print(tabulate(data,headers=col_names,tablefmt="fancy_grid"))

╒═════════════════════╤═══════════════╤══════════════╕
│ Model               │   Train score │   Test score │
╞═════════════════════╪═══════════════╪══════════════╡
│ KNN                 │      0.987821 │     0.986532 │
├─────────────────────┼───────────────┼──────────────┤
│ Logistic Regression │      0.928998 │     0.927313 │
├─────────────────────┼───────────────┼──────────────┤
│ Decision Tree       │      0.999909 │     0.997963 │
╘═════════════════════╧═══════════════╧══════════════╛


In [40]:
# # Chemin du fichier texte
# txt_file_path = './dataset/testData.txt'

# # Fonction pour ouvrir et lire le fichier texte
# def open_txt_file(file_path):
#     with open(file_path, 'r') as txt_file:
#         data = txt_file.read()
#     return data

# # Appel de la fonction pour ouvrir le fichier texte
# txt_data = open_txt_file(txt_file_path)
txt_data=pd.read_csv('captured_traffic.csv', sep=',')
# test=pd.read_csv('./dataset/testData.txt',sep=',')
# Affichage du contenu du fichier texte
# print(txt_data)
le(txt_data)

In [41]:
# txt_data.drop(['num_outbound_cmds'],axis=1,inplace=True)
txt_data


,src_ip,dst_ip,src_port,dst_port,protocol_type,service,flag,src_bytes,dst_bytes,count,same_srv_rate,diff_srv_rate,dst_host_srv_count,dst_host_same_srv_rate
0,5,4,65333,443,0,0,1,62,116,1,1.0,0.0,1,1.0
1,5,4,65333,443,0,0,1,161,215,1,1.0,0.0,2,1.0
2,5,4,65333,443,0,0,1,223,277,1,1.0,0.0,3,1.0
3,4,5,443,65333,0,1,1,0,54,1,1.0,0.0,1,1.0
4,4,5,443,65333,0,1,1,1400,1454,1,1.0,0.0,2,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6269,5,17,65460,443,0,0,1,1400,1454,1,1.0,0.0,1177,1.0
6270,5,17,65460,443,0,0,1,1400,1454,1,1.0,0.0,1178,1.0
6271,5,17,65460,443,0,0,1,1400,1454,1,1.0,0.0,1179,1.0
6272,5,17,65460,443,0,0,1,1400,1454,1,1.0,0.0,1180,1.0


In [42]:
train

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,1,20,9,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,1
1,0,2,44,9,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,1
2,0,1,49,5,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,0
3,0,1,24,9,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,1
4,0,1,24,9,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,1,49,5,0,0,0,0,0,0,...,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,0
125969,8,2,49,9,105,145,0,0,0,0,...,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,1
125970,0,1,54,9,2231,384,0,0,0,0,...,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,1
125971,0,1,30,5,0,0,0,0,0,0,...,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,0


In [43]:
train.columns

Index(['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'is_host_login', 'is_guest_login', 'count',
       'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate',
       'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate',
       'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count',
       'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
       'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
       'dst_host_serror_rate', 'dst_host_srv_serror_rate',
       'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'class'],
      dtype='object')

In [44]:
# txt_data.columns
selected_features

['protocol_type',
 'service',
 'flag',
 'src_bytes',
 'dst_bytes',
 'count',
 'same_srv_rate',
 'diff_srv_rate',
 'dst_host_srv_count',
 'dst_host_same_srv_rate']

In [45]:
newdata = txt_data[selected_features]
newdata=scale.fit_transform(newdata)
selected_features

['protocol_type',
 'service',
 'flag',
 'src_bytes',
 'dst_bytes',
 'count',
 'same_srv_rate',
 'diff_srv_rate',
 'dst_host_srv_count',
 'dst_host_same_srv_rate']

In [46]:
# # Sélection de l'en-tête
# entete = newdata[0, :]

# # Sélectionner une ligne spécifique (par exemple, la première ligne de données, pas l'en-tête)
# ligne = newdata[1, :]

# # Si vous voulez afficher cette ligne avec son "en-tête"
# # for nom_colonne, valeur in zip(entete, ligne):
# tableau_combine = np.vstack([entete, ligne])
# tableau_combine.shape

In [47]:
from joblib import load

# Charger le modèle à partir du fichier
dt_loaded = load('crssri.joblib')

# Nouvelles données pour prédiction
newdata
# Faire des prédictions
predictions = dt_loaded.predict(newdata)

print(predictions)  # Afficher les prédictions

[0 0 0 ... 1 1 0]
